# Scraping Ben & Jerry's ice cream flavors & reviews

Good to scrape
- Ben & Jerry's (include pints, Netflix Originals, cores, etc.)
   - Ben & Jerry's graveyard (no images or reviews)
- Haagen Dazs
- Breyers (no images)
- Talenti (no images)
- Dreyer's/Edy's (no images) -- seems to be an error in their website where reviews are mixed together for different flavors

Maybe
- Enlightened -- biased reviews
- Rebel -- biased reviews
- Wall's UK (Cornetto, Magnum,  Cart D'or) -- slow to load

Other
- Baskin Robbins, Graeter's, Halo Top, Blue Bell, Blue Bunny, Serendipity, So Delicious, Moevenpick, Alden's, Tillamook, Hood, Jeni's, Cold Stone, Gifford's, Brighams, Walpole Creamery, Friendly's, Nestle

# Requests

In [1]:
import requests
from lxml import html, etree
import time

# home = "https://www.benjerry.com/"
# url = home + "flavors/ice-cream-pints"
# req_main_page = requests.get(url)
# main_page = html.fromstring(req_main_page.content)
# products = response.xpath('//a[@class="landing-item"]')

NameError: name 'response' is not defined

In [4]:
for product in [products[0]]:    
    product_img_url = home + product.xpath('//img[@class="image-alt-name"]/@src')[0]
    
    req_product_page = requests.get(home + product.xpath("@data-producturl")[0])
    product_page = html.fromstring(req_product_page.content)
    
    product_name = product_page.xpath('//h1[@class="productDetails-name"]/text()')[0]
    product_subhead = product_page.xpath('//h2[@id="productDetails-product_desc"]/text()')[0].strip()
    product_story = product_page.xpath('//p[@id="productDetails-product_story"]/text()')[0].strip()
    
#     product_rating = product_page.xpath('//span[@itemprop="ratingValue"]/text()')
#     product_rating_ct = product_page.xpath('//span[@itemprop="reviewCount"]/text()')
#     product_img_url = product_page.xpath('//img[@class="lazy desktop lazy-stack loaded"]/@src')

    print(product_name, product_subhead, product_story)

/files/live/sites/systemsite/files/flavors/products/us/pint/open-closed-pints/boots-on-the-moooon-landing.png
Boots on the Moooo’n™ A Universe of Milk Chocolate Ice Cream with Fudge Cows & Toffee Meteor Clusters Orbiting a Sugar Cookie Dough Core While Space Force works to get boots (back) on the moon, this cosmic concoction will launch your dreams into orbit. With fudgy astronaut cows & toffee meteors, space couldn't be any sweeter. Unless the moon was made out of sugar cookie dough. Which for this limited-time flavor, it actually is!


# Selenium

In [1]:
from selenium import webdriver      
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd
import urllib.request
import json
import random
import numpy as np

# Ben & Jerry's

In [126]:
# initialize driver
driver = webdriver.Chrome('./chromedriver.exe')

# --- get starting page with list of products ---
# define starting page
home = "https://www.benjerry.com/"
url = home + "flavors/ice-cream-pints"

# load the page
driver.get(url)  
time.sleep(3)

# get the list of products
products = driver.find_elements_by_xpath('.//a[@class="landing-item"]')
product = products[0]
# --------------------------------------------

In [124]:
def go_to_product_page(product, driver, completed):
    product_name = product.get_attribute("data-pname")
    if product_name in completed:
        return True
    else:
        completed.append(product_name)
        product_page_url = home + product.get_attribute("data-producturl")
        driver.get(product_page_url)
        time.sleep(2) # wait for ratings to load
        return False

def get_product_page_content(driver):
    product_name = driver.find_element_by_xpath('.//h1[@class="productDetails-name"]').text.strip()
    print("Starting product:", product_name) 
    product_subhead = driver.find_element_by_xpath('.//h2[@id="productDetails-product_desc"]').text.strip()
    product_story = driver.find_element_by_xpath('.//p[@id="productDetails-product_story"]').text.strip()
    product_rating = driver.find_element_by_xpath('.//span[@itemprop="ratingValue"]').text
    product_rating_ct = driver.find_element_by_xpath('.//span[@itemprop="reviewCount"]').text
    
    ingred_expand = driver.find_element_by_xpath('.//li[@class="bjnt_flavorListItem"]/div[contains(@class,"accordion-basic")]')
    ingred_expand.click()
    time.sleep(2)
    try:
        ingred_info = driver.find_element_by_xpath('.//div[@class="package-ingredients"]').text.strip()
    except NoSuchElementException: 
        ingred_info = "NA"
    
#     print(product_name, product_subhead, product_story, product_rating, product_rating_ct)
    
    # filename for reviews data and image
    product_filename = "-".join(product_name.encode("ascii", "ignore").decode().replace('"','').strip().split())
    
    # product info
    df = pd.DataFrame({"key": [product_filename], "name": [product_name],
                       "subhead": [product_subhead], "description": [product_story],
                      "rating": [product_rating], "rating_count": [product_rating_ct],
                      "ingredients":[ingred_info]})
    
    if os.path.isfile("products_bj.csv"):
        df.to_csv("products_bj.csv", mode="a", header=False, index=False)
    else:
        df.to_csv("products_bj.csv", mode="w", header=True, index=False)
    
    return product_filename
    

def get_reviews_single_page(driver, product_filename):
    reviews_expand = driver.find_element_by_xpath('.//div[@id="expReviews"]')
    reviews_expand.click()
    time.sleep(4)
    
    # avoid the 2 review summary (most helpful, most harmful) and avoid review responses 
    reviews_single_page = driver.find_elements_by_xpath('.//div[contains(@class,"bv-content-item-avatar-offset") and not(ancestor::div[@class="bv-section-summary"] or ancestor::li[contains(@class,"bv-secondary-content-clientresponse")])]')
    
    review_author, review_date, review_stars, review_title, review_yes, review_no, review_text = [], [], [], [], [], [], []
    for review in reviews_single_page:
        try:
            review_author.append(review.find_element_by_xpath('.//button[@itemprop="author"]/span').text)
        except NoSuchElementException: 
            review_author.append("NA") # anonymous
        review_date.append(review.find_element_by_xpath('.//meta[@itemprop="datePublished"]').get_attribute("content"))
        review_stars.append(review.find_element_by_xpath('.//meta[@itemprop="ratingValue"]').get_attribute("content"))
        try:
            review_title.append(review.find_element_by_xpath('.//h3[@itemprop="headline"]').text)
        except NoSuchElementException: 
            review_title.append("NA")
        try:
            review_yes.append(review.find_element_by_xpath('.//button[contains(@class,"feedback-yes")]//span[@class="bv-content-btn-count"]').text)
            review_no.append(review.find_element_by_xpath('.//button[contains(@class,"feedback-no")]//span[@class="bv-content-btn-count"]').text)
        except NoSuchElementException:
            review_yes.append("0")
            review_no.append("0")
        review_text.append(review.find_element_by_xpath('.//div[@class="bv-content-summary-body-text"]/p').text)
            
    product_filename_column = [product_filename]*len(reviews_single_page) 
    df = pd.DataFrame({"key": product_filename_column, "author": review_author, "date": review_date, 
                       "stars": review_stars, "title": review_title, "helpful_yes": review_yes, "helpful_no": review_no, 
                       "text": review_text})

    if os.path.isfile("reviews_bj.csv"):
        df.to_csv("reviews_bj.csv", mode="a", header=False, index=False)
    else:
        df.to_csv("reviews_bj.csv", mode="w", header=True, index=False)
        
    print("Processed", len(reviews_single_page), "reviews")
        
def pagination(driver):
    try:
        next_page = driver.find_element_by_xpath('.//a[descendant::span[@class="bv-content-btn-pages-next"]]') 
    except NoSuchElementException: 
        more = False
    else:
        more = True
        next_page.click()
        time.sleep(2)
        
    return more # True means there is another page to scrape

In [127]:
# make directory for images and reviews
if not os.path.isdir("images_bj"):
    os.mkdir("images_bj")

# get list of images (circumvent lazy load)
product_img_urls = []
for prod in products:
    driver.execute_script("arguments[0].scrollIntoView()", prod)
    try:
        photo = prod.find_element_by_xpath('.//img[@class="image-alt-name"]')
    except NoSuchElementException:
        print("Photo not found") 
        product_img_urls.append("NA")
    else:
        product_img_urls.append(photo.get_attribute("src"))

i = 0
df1=pd.read_csv("products_bj.csv")
completed = df1["name"].values.tolist()
# completed = []
while i < len(products):
    isCompleted = go_to_product_page(product, driver, completed)
    if isCompleted:
        i += 1
        product = driver.find_elements_by_xpath('.//a[@class="landing-item"]')[i]
        continue
        
    product_filename = get_product_page_content(driver)
        
    # get all reviews for product
    nextPage = True
    while nextPage == True:
        get_reviews_single_page(driver, product_filename)
        nextPage = pagination(driver)
    
    # get product image
    product_img_url = product_img_urls[i]
    if product_img_url != "NA":
        urllib.request.urlretrieve(product_img_url, "images_bj/%s.png" % product_filename)      
    time.sleep(2.5)
        
    # return to start
    driver.get(url)
    i += 1
    product = driver.find_elements_by_xpath('.//a[@class="landing-item"]')[i]

Photo not found
Starting product: Cold Brew Caramel Latte
Processed 8 reviews
Processed 2 reviews
Starting product: Wake & " No Bake " Cookie Dough Core
Processed 8 reviews
Processed 30 reviews
Processed 30 reviews
Starting product: Brownie Batter Core
Processed 8 reviews
Processed 30 reviews
Processed 30 reviews
Processed 30 reviews
Processed 10 reviews
Starting product: Cookies & Cream Cheesecake Core
Processed 8 reviews
Processed 30 reviews
Processed 19 reviews
Starting product: Karamel Sutra® Core
Processed 8 reviews
Processed 30 reviews
Processed 30 reviews
Processed 26 reviews
Starting product: Peanut Butter Fudge Core
Processed 8 reviews
Processed 11 reviews


IndexError: list index out of range

# Haagen-Dazs

In [189]:
# initialize driver
driver = webdriver.Chrome('./chromedriver.exe')

# --- get starting page with list of products ---
# define starting page
home = "https://www.haagendazs.us/"
url = home + "products"

# load the page
driver.get(url)  
time.sleep(3)

# get the list of products
products = driver.find_elements_by_xpath('//div[contains(@class,"flavors")]')
product = products[0]
# --------------------------------------------

In [190]:
def go_to_product_page(product, driver, completed):
    product_name = product.find_element_by_xpath('.//div[@class="name-product"]//span').text.strip()
    if product_name in completed:
        return True
    else:
        completed.append(product_name)
        product_page_url = product.find_element_by_xpath('.//a').get_attribute("href")
        driver.get(product_page_url)
        time.sleep(2.5) # wait for ratings to load
        return False

def get_product_page_content(driver):
    product_name = driver.find_element_by_xpath('.//div[@itemprop="name"]/h1').text.strip()
    print("Starting product:", product_name) 
    product_description = driver.find_element_by_xpath('.//span[@itemprop="description"]//p').text.strip()
    product_rating = driver.find_element_by_xpath('.//div[@itemprop="ratingValue"]').text
    product_rating_ct = driver.find_element_by_xpath('.//meta[@itemprop="reviewCount"]').get_attribute("content")
    
    # ------ INGREDIENTS ---------
    fail = False
    # normal label
    try:
        ingred_expand = driver.find_element_by_xpath('.//a[@class="nutritional-table"]')
        ingred_expand.click()
        time.sleep(0.5)
    except NoSuchElementException:
        fail = True
    else:
        try:
            ingred_info = driver.find_element_by_xpath('.//div[@class="ingredients"]/p').text.lstrip("Ingredients:  ").split(" Contains:")[0]
        except NoSuchElementException: 
            ingred_info = "NA"
        finally:
            ingred_close = driver.find_element_by_xpath('.//a[@class="close"]')
            ingred_close.click()
            time.sleep(0.5)
    # smart-label
    if fail == True:
        try:
            ingred_expand = driver.find_element_by_xpath('.//a[@class="info-smart-label-url"]')
        except NoSuchElementException: 
            ingred_info = "NA"
        else:
            ingred_expand.click()
            time.sleep(2)
            
            try:
                frame = driver.find_element_by_xpath('.//iframe[contains(@src,"smartlabel")]')
            except NoSuchElementException: 
                ingred_info = "NA"
            else:
                time.sleep(0.5)
#                 WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,'.//iframe[contains(@src,"smartlabel")]'')))
                driver.switch_to.frame(frame)

                ingred_nav = driver.find_element_by_xpath('.//div[contains(@class,"TabNavigation")]//span[contains(text(),"Ingredients")]')
                ingred_nav.click()
                time.sleep(0.5)

                ingred_list = driver.find_elements_by_xpath('.//div[contains(@class,"IngredientList")]//a')
                ingred_info = ", ".join([ingred.text for ingred in ingred_list])

                driver.switch_to.default_content()
                ingred_close = driver.find_element_by_xpath('.//a[@class="modal-vanilla-close"]')
                ingred_close.click()
                time.sleep(0.5)
    # -----------------------------
    
    # filename for reviews data and image
    product_filename = "-".join(product_name.encode("ascii", "ignore").decode().strip().split())
    
    # product info
    df = pd.DataFrame({"key": [product_filename], "name": [product_name],
                       "description": [product_description],
                      "rating": [product_rating], "rating_count": [product_rating_ct],
                      "ingredients": [ingred_info]})
    
    if os.path.isfile("products_hd.csv"):
        df.to_csv("products_hd.csv", mode="a", header=False, index=False)
    else:
        df.to_csv("products_hd.csv", mode="w", header=True, index=False)    
    
    # get product image
    product_img_url = driver.find_element_by_xpath('.//div[@class="hero-details"]//img').get_attribute("src")
    urllib.request.urlretrieve(product_img_url, "images_hd/%s.png" % product_filename)      
    time.sleep(2)
    
    return product_filename
    

def get_reviews_single_page(driver, product_filename):
    time.sleep(3) # wait for reviews to load
    
    reviews_single_page = driver.find_elements_by_xpath('.//li[contains(@class,"bv-content-review")]')
    
    review_author, review_date, review_stars, review_title, review_yes, review_no, review_text = [], [], [], [], [], [], []
    tastes, ingredients, textures, likes = [], [], [], []
    for review in reviews_single_page:
        try:
            review_author.append(review.find_element_by_xpath('.//div[@class="bv-author-avatar"]//h3').text)
        except NoSuchElementException: 
            review_author.append("NA") # anonymous
        review_date.append(review.find_element_by_xpath('.//meta[@itemprop="datePublished"]').get_attribute("content"))
        review_stars.append(review.find_element_by_xpath('.//meta[@itemprop="ratingValue"]').get_attribute("content"))
        try:
            review_title.append(review.find_element_by_xpath('.//h3[@itemprop="headline"]').text)
        except NoSuchElementException: 
            review_title.append("NA")
        try:
            review_yes.append(review.find_element_by_xpath('.//button[contains(@class,"feedback-yes")]//span[@class="bv-content-btn-count"]').text)
            review_no.append(review.find_element_by_xpath('.//button[contains(@class,"feedback-no")]//span[@class="bv-content-btn-count"]').text)
        except NoSuchElementException:
            review_yes.append("0")
            review_no.append("0")
        review_text.append(review.find_element_by_xpath('.//div[@class="bv-content-summary-body-text"]/p').text)
        
        # misc   
        try:
            secondary_labels = review.find_elements_by_xpath('.//li[@class="bv-content-secondary-ratings-label"]')
        except NoSuchElementException: 
            tastes.append("NA")
            ingredients.append("NA")
            textures.append("NA")

        else:
            if len(secondary_labels) == 0:
                tastes.append("NA")
                ingredients.append("NA")
                textures.append("NA")
            else:
                for label in secondary_labels:
                    label_text = label.text
                    rating_bar = label.find_element_by_xpath('./following-sibling::li[1]')
                    rating_span = rating_bar.find_element_by_xpath('.//span[contains(@class,"bv-content-secondary-ratings-value")]')
                    rating0_100 = rating_span.get_attribute("class").split("-")[-1] # this is a number between 0 and 100
                    rating1_5 = int(rating0_100)/20

                    if label_text == "Overall Taste":
                        tastes.append(rating1_5)
                    elif label_text == "Quality of Ingredients":
                        ingredients.append(rating1_5)
                    elif label_text == "Overall Texture":
                        textures.append(rating1_5)
                    
        try:
            dimensions = review.find_elements_by_xpath('.//ul[@class="bv-content-data-tag-dimensions"]//li[@class="bv-content-data-value"]')
        except NoSuchElementException: 
            likes.append("NA")
        else:
            if len(dimensions) == 0:
                likes.append("NA")
            else:
                likes.append("".join([d.text for d in dimensions]))
            
    product_filename_column = [product_filename]*len(reviews_single_page) 
    df = pd.DataFrame({"key": product_filename_column, "author": review_author, "date": review_date, 
                       "stars": review_stars, "title": review_title, "helpful_yes": review_yes, "helpful_no": review_no, 
                       "text": review_text, "taste": tastes, "ingredients":ingredients, "texture": textures, "likes": likes})

    if os.path.isfile("reviews_hd.csv"):
        df.to_csv("reviews_hd.csv", mode="a", header=False, index=False)
    else:
        df.to_csv("reviews_hd.csv", mode="w", header=True, index=False)
        
    print("Processed", len(reviews_single_page), "reviews")
        
def pagination(driver):
    try:
        next_page = driver.find_element_by_xpath('.//button[contains(@class,"bv-content-btn-pages-load-more")]') 
    except NoSuchElementException: 
        more = False
    else:
        more = True
        driver.execute_script("arguments[0].scrollIntoView()", next_page)
        next_page.click()
        time.sleep(1)
        
    return more # True means there is another page to scrape

In [191]:
# make directory for images and reviews
if not os.path.isdir("images_hd"):
    os.mkdir("images_hd")

# load into view
for prod in products:
    driver.execute_script("arguments[0].scrollIntoView()", prod)
    
i = 0
df1=pd.read_csv("products_hd.csv")
completed = df1["name"].values.tolist()
while i < len(products):
    print(i)
    isCompleted = go_to_product_page(product, driver, completed)
    if isCompleted:
        i += 1
        product = driver.find_elements_by_xpath('.//div[contains(@class,"flavors")]')[i]
        continue
        
    product_filename = get_product_page_content(driver)
        
    # get all reviews for product
    nextPage = True
    while nextPage == True:
        nextPage = pagination(driver)
    get_reviews_single_page(driver, product_filename)    
    
    # return to start
    driver.get(url)
    i += 1
    product = driver.find_elements_by_xpath('.//div[contains(@class,"flavors")]')[i]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Starting product: Chocolate Fudge Non-Dairy Bar
Processed 22 reviews
15
Starting product: Chocolate Peanut Butter Ice Cream
Processed 173 reviews
16
Starting product: Chocolate Salted Fudge Truffle Non-Dairy
Processed 97 reviews
17
Starting product: Chocolate Sea Salt Caramel HEAVEN Light Ice Cream
Processed 13 reviews
18
Starting product: Coconut Caramel Non-Dairy
Processed 11 reviews
19
Starting product: Coconut Caramel Chocolate TRIO CRISPY LAYERS
Processed 26 reviews
20
Starting product: Coconut Caramel Dark Chocolate Non-Dairy Bar
Processed 31 reviews
21
Starting product: Coffee Ice Cream
Processed 173 reviews
22
Starting product: Coffee Almond Crunch Ice Cream Bar
Processed 42 reviews
23
Starting product: Coffee Chip Ice Cream
Processed 111 reviews
24
Starting product: Coffee Vanilla Chocolate TRIO CRISPY LAYERS
Processed 48 reviews
25
Starting product: Cold Brew Espresso Chip HEAVEN Light Ice Cream
Processed 18 reviews
26
Starting product: Cook

IndexError: list index out of range

# Talenti

In [100]:
# initialize driver
driver = webdriver.Chrome('./chromedriver.exe')

# --- get starting page with list of products ---
# define starting page
home = "https://www.talentigelato.com/"
url = home + "product-category/talenti-gelato-flavors"

# load the page
driver.get(url)  
time.sleep(3)

# get the list of products
products = driver.find_elements_by_xpath('//a[@data-product-id]')
i = 0 # starting product
product = products[i]

# close popup
close_popup = driver.find_element_by_xpath('.//button[@class="close" and @aria-hidden="true"]')
close_popup.click()
time.sleep(1.5)

There are 4 different formats for product description. They are all found under `<div class="the_content">`.

1.
```HTML
    <div class="left*"> </div>
    <div class="right*"> </div>
    <div class="pdp-recipe"> </div>
    <div> </div> <!-- empty div as a separator -->
    <div class="pdp-recipe"> ... </div> <!-- text description -->
...
```

2. 
```HTML
    <div class="pdp-recipe"> </div>
    <div class="description">
        <p> ... </p> <!-- text description -->
    </div>
...
```

3. 
```HTML
    <div class="left*"> </div>
    <div class="right*"> </div>
    <p> ... </p> <!-- text description -->
...
```

4.
```HTML
    <div class="left*"> </div>
    <div class="right*"> </div>
    <div class="description"> ... </div> <!-- text description -->
...
```

In [101]:
def go_to_product_page(product, driver, completed):
    product_name = product.get_attribute('data-ct-information').strip()
    if product_name in completed:
        return True
    else:
        completed.append(product_name)
        product_page_url = product.get_attribute("href")
        driver.get(product_page_url)
        time.sleep(4) # wait for ratings to load
        return False

def get_product_page_content(driver):
    product_name = driver.find_element_by_xpath('.//div[@class="product-description"]//h2').text.strip()
    print("Starting product:", product_name) 
    with open("last_started.txt", "w") as f:
        f.write(product_name)
    # ---- PRODUCT DESCRIPTION: 4 different formats -----
    pd_fmt1 = driver.find_elements_by_xpath('.//div[@class="the_content"]//div[@class="pdp-recipe"]')
    if len(pd_fmt1) == 1:
        try:
            product_description = driver.find_element_by_xpath('.//div[@class="the_content"]//div[@class="description"]/p').text
        except NoSuchElementException:
            product_description = pd_fmt1[0].text
    elif len(pd_fmt1) == 2:
        product_description = pd_fmt1[-1].text
    elif len(pd_fmt1) == 0:
        try:
            pd_fmt2 = driver.find_element_by_xpath('.//div[child::div[contains(@id,"left")] and child::div[contains(@id,"right")]]/p')
        except NoSuchElementException:
            try:
                pd_fmt3 = driver.find_element_by_xpath('.//div[child::div[contains(@id,"left")] and child::div[contains(@id,"right")]]/div[@class="description"]')
            except NoSuchElementException:
                product_description = "NA"
            else:
                product_description = pd_fmt3.text
        else:
            product_description = pd_fmt2.text

    else:
        product_description = "NA"
    # ---------------------------------------------
    # --- RATING AND RATING COUNT ---
    # Some products are formatted differently and do not show rating value and rating count, however we can impute them
    # from looking at the review data
    try:
        product_rating = driver.find_element_by_xpath('.//div[@itemprop="ratingValue"]').text
    except NoSuchElementException:
        product_rating = "NA"
    try:
        product_rating_ct = driver.find_element_by_xpath('.//meta[@itemprop="reviewCount"]').get_attribute("content")
    except NoSuchElementException:
        product_rating_ct = "NA"
    
    # ------ INGREDIENTS ---------
    ingred_info = driver.find_element_by_xpath('.//div[@class="product-ingredients"]').text.lstrip("INGREDIENTS").split("May Contain")[0].strip().rstrip(".").upper()
    
    # filename for reviews data and image
    product_filename = "-".join(product_name.encode("ascii", "ignore").decode().strip().split())
    
    # product info
    df = pd.DataFrame({"key": [product_filename], "name": [product_name],
                       "description": [product_description],
                      "rating": [product_rating], "rating_count": [product_rating_ct],
                      "ingredients": [ingred_info]})
    
    if os.path.isfile("products_talenti.csv"):
        df.to_csv("products_talenti.csv", mode="a", header=False, index=False)
    else:
        df.to_csv("products_talenti.csv", mode="w", header=True, index=False)    
    
    # get product image
    if not os.path.isfile("images_talenti/%s.png" % product_filename):
        product_img_url = driver.find_element_by_xpath('.//img[@itemprop="image"]').get_attribute("src")
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-agent', 'Chrome/85.0.4183.121')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(product_img_url, "images_talenti/%s.png" % product_filename)      
        time.sleep(4)
    
    return product_filename
    
def get_reviews_single_page(driver, product_filename):
    reviews_single_page = driver.find_elements_by_xpath('.//li[contains(@class,"bv-content-review")]')
    
    review_author, review_date, review_stars, review_title, review_yes, review_no, review_text = [], [], [], [], [], [], []
    for review in reviews_single_page:
        try:
            review_author.append(review.find_element_by_xpath('.//button[@itemprop="author"]/span').text)
        except NoSuchElementException: 
            review_author.append("NA") # anonymous
        review_date.append(review.find_element_by_xpath('.//meta[@itemprop="datePublished"]').get_attribute("content"))
        review_stars.append(review.find_element_by_xpath('.//meta[@itemprop="ratingValue"]').get_attribute("content"))
        try:
            review_title.append(review.find_element_by_xpath('.//h3[@itemprop="headline"]').text)
        except NoSuchElementException: 
            review_title.append("NA")
        try:
            review_yes.append(review.find_element_by_xpath('.//button[contains(@class,"feedback-yes")]//span[@class="bv-content-btn-count"]').text)
            review_no.append(review.find_element_by_xpath('.//button[contains(@class,"feedback-no")]//span[@class="bv-content-btn-count"]').text)
        except NoSuchElementException:
            review_yes.append("0")
            review_no.append("0")
        paragraphs = review.find_elements_by_xpath('.//div[@class="bv-content-summary-body-text"]//p')
        review_text.append("\n".join([p.text for p in paragraphs]))
        
    product_filename_column = [product_filename]*len(reviews_single_page) 
    df = pd.DataFrame({"key": product_filename_column, "author": review_author, "date": review_date, 
                       "stars": review_stars, "title": review_title, "helpful_yes": review_yes, "helpful_no": review_no, 
                       "text": review_text})

    if os.path.isfile("reviews_talenti.csv"):
        df.to_csv("reviews_talenti.csv", mode="a", header=False, index=False)
    else:
        df.to_csv("reviews_talenti.csv", mode="w", header=True, index=False)
        
    print("Processed", len(reviews_single_page), "reviews")
        
def pagination(driver):
       
    try:
        next_page = driver.find_element_by_xpath('.//span[@class="bv-content-btn-pages-next"]') 
    except NoSuchElementException: 
        more = False
    else: 
        xofy = driver.find_elements_by_xpath('.//div[contains(@class,"bv-content-pagination-pages-current")]/span')[-1].text.strip()
        xofy_text = xofy.split() # example: xofy_text = "99–114 of 114 Reviews        &nbsp;".split()
        page_end = xofy_text[0].split("–")[1] # xofy_text[0] = 99-114, xofy_text.split("-")[1] = 114
        total = xofy_text[2] # total = 114
        if page_end == total:
            more = False
        else:
            more = True
            
            driver.execute_script("arguments[0].scrollIntoView()", next_page)
            time.sleep(3)
            next_page.click()
            time.sleep(2)
        
    return more # True means there is another page to scrape

In [102]:
# make directory for images and reviews
if not os.path.isdir("images_talenti"):
    os.mkdir("images_talenti")

# load into view
for prod in products:
    driver.execute_script("arguments[0].scrollIntoView()", prod)
    
# -- FIRST RUN --
# completed = []
# -- RESUMING --
df1=pd.read_csv("products_talenti.csv")
completed = df1["name"].values.tolist()

while i < len(products):
    print(i)
    isCompleted = go_to_product_page(product, driver, completed)
    if isCompleted:
        i += 1
        product = driver.find_elements_by_xpath('.//a[@data-product-id]')[i]
        continue
        
    product_filename = get_product_page_content(driver)
        
    # --- REVIEWS ---
    # close popup
    try:
        close_popup = driver.find_element_by_xpath('.//span[@id="closedelivery"]')
    except NoSuchElementException:
        pass
    else:
        close_popup.click()
        time.sleep(0.5)
    
    # format 1: clickable review tab
    try:
        open_reviews = driver.find_element_by_xpath('.//a[@data-ct-information="Reviews"]')
    
    # format 2: no click needed
    except NoSuchElementException:
        review_sec = driver.find_element_by_xpath('.//div[@class="heading_h4"]/span[contains(text(),"Reviews")]')
        driver.execute_script("arguments[0].scrollIntoView()", review_sec)
    
    # format 1 
    else:
        # scroll to reviews tab
        driver.execute_script("arguments[0].scrollIntoView()", open_reviews)
        time.sleep(4 + random.random()*2 )
        try:
            # open the reviews tab
            open_reviews.click()
            time.sleep(14 + random.random()*2 ) # wait for reviews to load
        except: # extra scrolling may be needed
            pass
            
    # both formats
    finally:
        # get all reviews for product
        nextPage = True
        while nextPage == True:
            get_reviews_single_page(driver, product_filename)  
            nextPage = pagination(driver)
            
    
    # return to start
    driver.get(url)
    i += 1
    product = driver.find_elements_by_xpath('.//a[@data-product-id]')[i]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
Starting product: VANILLA FUDGE COOKIE
Processed 8 reviews
Processed 30 reviews
Processed 30 reviews
Processed 30 reviews
Processed 16 reviews


IndexError: list index out of range

In [99]:
# if failed search on product, call this function to remove data before resuming
def remove_data_pname(product_name):
    key = "-".join(product_name.encode("ascii", "ignore").decode().strip().split())
    
    df = pd.read_csv("products_talenti.csv")
    df[df["key"] != key].to_csv("products_talenti.csv", index=False)
    
    df = pd.read_csv("reviews_talenti.csv")
    df[df["key"] != key].to_csv("reviews_talenti.csv", index=False)
    
def clean_up():
    with open("last_started.txt", "r") as f:
        product_name = f.read()
        remove_data_pname(product_name)

remove_data_pname("VANILLA FUDGE COOKIE")

In [146]:
prod.head()

,key,name,description,rating,rating_count,ingredients
0,ALPHONSO-MANGO-SORBETTO,ALPHONSO MANGO SORBETTO,Our simple and delicious Alphonso Mango Sorbet...,4.7,139.0,"MANGOS, WATER, SUGAR, DEXTROSE, LEMON JUICE, C..."
1,BANANA-CARAMEL-CRUNCH,BANANA CARAMEL CRUNCH,This flavor was inspired by a classic southern...,4.2,25.0,"MILK, BANANAS, SUGAR†, WHEAT FLOUR, CREAM, NON..."
2,BELGIAN-CHOCOLATE-GELATO,BELGIAN CHOCOLATE GELATO,Our Belgian Chocolate gelato is made with melt...,4.8,27.0,"MILK, SUGAR, CREAM, EGG AND EGG YOLK, DEXTROSE..."
3,BLACK-RASPBERRY-CHOCOLATE-CHIP-GELATO,BLACK RASPBERRY CHOCOLATE CHIP GELATO,"Tart and sweet, our Black Raspberry Chocolate ...",4.0,105.0,"MILK, SUGAR, CREAM, BLACK RASPBERRIES, DEXTROS..."
4,BLACK-RASPBERRY-VANILLA-PARFAIT,BLACK RASPBERRY VANILLA PARFAIT,This Layer was inspired by one of our favorite...,4.5,65.0,"MILK, SUGAR, CREAM, BLACK RASPBERRY PUREE, ROL..."


In [167]:
prod = pd.read_csv("products_talenti.csv")
rev = pd.read_csv("reviews_talenti.csv")

print(prod.shape, rev.shape)

# remove duplicates
print(prod.duplicated().sum(), rev.duplicated().sum())
prod = prod[~prod.duplicated()]
rev = rev[~rev.duplicated()] 

# impute missing rating & rating count by referring to reviews
print(prod.isna().sum()) 
na_locs = prod[["rating","rating_count"]].isna().any(axis=1)
prod_na = prod[na_locs]
rev_gp = rev[["key","stars"]].groupby("key")
m = rev_gp.mean()
c = rev_gp.count() 
for key in prod_na["key"].values:
    prod.loc[prod["key"]==key, "rating"] = np.round(m.loc[m.index==key, "stars"].values, 1)
    prod.loc[prod["key"]==key, "rating_count"] = c.loc[c.index==key, "stars"].values
prod["rating_count"] = prod["rating_count"].astype('int32')
    
display(prod[na_locs].head())

prod.to_csv("products_talenti.csv", index=False)
rev.to_csv("reviews_talenti.csv", index=False)

(45, 6) (4074, 8)
0 5
key             0
name            0
description     4
rating          4
rating_count    4
ingredients     0
dtype: int64


,key,name,description,rating,rating_count,ingredients
14,COFFEE-COOKIE-CRUMBLE,COFFEE COOKIE CRUMBLE,"Inspired by an affogato, Coffee Cookie Crumble...",4.4,26,"MILK, SUGAR†, CREAM, CHOCOLATE, EGG YOLKS, WAT..."
22,LEMON-BERRY-PIE,LEMON BERRY PIE,"Inspired by a classic summer dessert, this Lay...",4.2,31,"MILK, SUGAR†, CREAM, LEMON JUICE, WHEAT FLOUR,..."
25,MINT-FUDGE-COOKIE,MINT FUDGE COOKIE,"In this Layer, our Mediterranean Mint Gelato i...",4.7,52,"MILK, SUGAR, CREAM, COCONUT OIL, WHEAT FLOUR, ..."
41,STRAWBERRY-HIBISCUS-SORBETTO,STRAWBERRY HIBISCUS SORBETTO,Sweet strawberries and real hibiscus flowers a...,4.4,14,"WATER, STRAWBERRIES, SUGAR, DEXTROSE, CORN SYR..."


In [111]:
rev[~rev.duplicated()].shape, rev[rev.duplicated()].shape

((4069, 8), (5, 8))

# Breyer's

In [39]:
# initialize driver
driver = webdriver.Chrome('./chromedriver.exe')

# initialize headers for urllib.request
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Chrome/85.0.4183.121')]
urllib.request.install_opener(opener)

# --- get starting page with list of products ---
# define starting page
home = "https://www.breyers.com"
url = home + "/us/en/products.html"

# load the page
driver.get(url)  
time.sleep(3)

# close popup
close_popup = driver.find_element_by_xpath('.//a[@class="button-close"]')
close_popup.click()
time.sleep(1.5)

# -------- get the list of products ----------
def show_more(driver):
    more_products = True
    while more_products:
        btn = driver.find_element_by_xpath('.//button[@data-limit]')
        driver.execute_script("arguments[0].scrollIntoView()", btn)
        time.sleep(1.5)
        driver.execute_script("window.scrollBy(0,-200);") # navbar offset
        time.sleep(1.5)
        try:
            btn.click()
        except: # button no longer clickable ==> no more products
            more_products = False
        else:
            time.sleep(3)
show_more(driver)

products = driver.find_elements_by_xpath('//div[contains(@class,"box listing-item")]')
i = 0
product = products[i]
# --------------------------------------------

Scrap work

In [7]:
# # initialize driver
# driver = webdriver.Chrome('./chromedriver.exe')
# url = "https://www.breyers.com/us/en/products/classics/natural-vanilla.html"

# # load the page
# driver.get(url)  
# time.sleep(3)

# # close popup
# close_popup = driver.find_element_by_xpath('.//a[@class="button-close"]')
# close_popup.click()
# time.sleep(1.5)

In [40]:
def go_to_product_page(product, driver, completed):
    product_name = product.find_element_by_xpath('.//div[@class="richText-content"]/h3/a').text.rstrip("&nbsp;").strip()
    if product_name in completed:
        return True
    else:
        completed.append(product_name)
        product_page_url = product.find_element_by_xpath('.//div[@class="richText-content"]/h3/a').get_attribute("href")
        driver.get(product_page_url)
        time.sleep(2.5) # wait for ratings to load
        return False

def get_product_page_content(driver):
    product_name = driver.find_elements_by_xpath('.//div[@class="richText-content"]/h1')[0].text.split("\n")[0]
    print("Starting product:", product_name)
    with open("last_started.txt", "w") as f:
        f.write(product_name)
    product_description = driver.find_elements_by_xpath('.//div[@class="richText-content"]/p/span')[0].text.strip()
    product_rating = driver.find_element_by_xpath('.//div[@itemprop="ratingValue"]').text
    product_rating_ct = driver.find_element_by_xpath('.//meta[@itemprop="reviewCount"]').get_attribute("content")
    
    # filename for reviews data and image
    product_filename = "-".join(product_name.encode("ascii", "ignore").decode().strip().split())  

    # get product image
    if not os.path.isfile("images_breyers/%s.png" % product_filename):
        product_img = driver.find_elements_by_xpath('.//div[@class="imageGallery-view"]//img')[0]
        product_img_url = product_img.get_attribute("src")
        if not product_img_url:
            product_img_url = product_img.get_attribute("data-src")
        if not product_img_url.startswith("https://www.breyers.com"):
            product_img_url = "https://www.breyers.com" + product_img_url
        urllib.request.urlretrieve(product_img_url, "images_breyers/%s.png" % product_filename)     
        time.sleep(4.5)
    
    # ------ INGREDIENTS ---------
    # smart-label
    try:
        sl = driver.find_element_by_xpath('.//div[@class="smartlabel-button"]//a')
    except NoSuchElementException: 
        ingred_info = "NA"
    else:
        driver.execute_script("arguments[0].scrollIntoView()", sl)
        time.sleep(0.5)
        driver.get(sl.get_attribute("href"))
        time.sleep(2.5)


        ingred_nav = driver.find_element_by_xpath('.//div[contains(@class,"TabNavigation")]//span[contains(text(),"Ingredients")]')
        ingred_nav.click()
        time.sleep(0.5)

        ingred_list = driver.find_elements_by_xpath('.//div[contains(@class,"IngredientList")]//a')
        ingred_info = ", ".join([ingred.text.upper() for ingred in ingred_list])
        
        # back to product page
        driver.back()
        time.sleep(1)
        driver.back()
        time.sleep(1)
    # -----------------------------
            
    # product info
    df = pd.DataFrame({"key": [product_filename], "name": [product_name],
                       "description": [product_description],
                      "rating": [product_rating], "rating_count": [product_rating_ct],
                      "ingredients": [ingred_info]})
    
    if os.path.isfile("products_breyers.csv"):
        df.to_csv("products_breyers.csv", mode="a", header=False, index=False)
    else:
        df.to_csv("products_breyers.csv", mode="w", header=True, index=False)  
    
    return product_filename
    

def get_reviews_single_page(driver, product_filename):
    time.sleep(3) # wait for reviews to load
    
    reviews_single_page = driver.find_elements_by_xpath('.//li[contains(@class,"bv-content-review")]')
    
    review_author, review_date, review_stars, review_title, review_yes, review_no, review_text = [], [], [], [], [], [], []
    for review in reviews_single_page:
        try:
            review_author.append(review.find_element_by_xpath('.//span[@class="bv-author"]').text)
        except NoSuchElementException: 
            review_author.append("NA") # anonymous
        review_date.append(review.find_element_by_xpath('.//meta[@itemprop="datePublished"]').get_attribute("content"))
        review_stars.append(review.find_element_by_xpath('.//meta[@itemprop="ratingValue"]').get_attribute("content"))
        try:
            review_title.append(review.find_element_by_xpath('.//h3[@itemprop="headline"]').text.strip())
        except NoSuchElementException: 
            review_title.append("NA")
        try:
            review_yes.append(review.find_element_by_xpath('.//button[contains(@class,"feedback-yes")]//span[@class="bv-content-btn-count"]').text)
            review_no.append(review.find_element_by_xpath('.//button[contains(@class,"feedback-no")]//span[@class="bv-content-btn-count"]').text)
        except NoSuchElementException:
            review_yes.append("0")
            review_no.append("0")
        review_paragraphs = review.find_elements_by_xpath('.//div[@class="bv-content-summary-body-text"]')[0]
        review_text.append( "\n".join([p.text for p in review_paragraphs.find_elements_by_xpath('./p')]) )
            
    product_filename_column = [product_filename]*len(reviews_single_page) 
    df = pd.DataFrame({"key": product_filename_column, "author": review_author, "date": review_date, 
                       "stars": review_stars, "title": review_title, "helpful_yes": review_yes, "helpful_no": review_no, 
                       "text": review_text})

    if os.path.isfile("reviews_breyers.csv"):
        df.to_csv("reviews_breyers.csv", mode="a", header=False, index=False)
    else:
        df.to_csv("reviews_breyers.csv", mode="w", header=True, index=False)
        
    print("Processed", len(reviews_single_page), "reviews")
        
def pagination(driver):
       
    try:
        next_page = driver.find_element_by_xpath('.//span[@class="bv-content-btn-pages-next"]') 
    except NoSuchElementException: 
        more = False
        print("Next page not found")
    else: 
        xofy = driver.find_elements_by_xpath('.//div[contains(@class,"bv-content-pagination-pages-current")]/span')[-1].text.strip()
        xofy_text = xofy.split() # example: xofy_text = "99–114 of 114 Reviews        &nbsp;".split()
        total = xofy_text[2] # total = 114
        try: # format 1: last page reads "99-114 of 114 Reviews"
            page_end = xofy_text[0].split("–")[1] # xofy_text[0] = 99-114, xofy_text.split("-")[1] = 114
        except: # format 2: last page reads "114 of 114 Reviews"
            page_end = xofy_text[0]
        if page_end == total:
            more = False
        else:
            more = True
            
            driver.execute_script("arguments[0].scrollIntoView()", next_page)
            time.sleep(2)
            driver.execute_script("window.scrollBy(0,-200);") # navbar offset
            time.sleep(1.5)
            next_page.click()
#             time.sleep(2)
        
    return more # True means there is another page to scrape

In [41]:
# make directory for images and reviews
if not os.path.isdir("images_breyers"):
    os.mkdir("images_breyers")

# ----- load into view -----
for prod in products:
    driver.execute_script("arguments[0].scrollIntoView()", prod)
# -------------------------

# -- FIRST RUN --
# completed = []
# -- RESUMING --
df1=pd.read_csv("products_breyers.csv")
completed = df1["name"].values.tolist()

while i < len(products):
    print(i)
    isCompleted = go_to_product_page(product, driver, completed)
    if isCompleted:
        i += 1
        product = driver.find_elements_by_xpath('.//div[contains(@class,"box listing-item")]')[i]
        continue
        
    product_filename = get_product_page_content(driver)
    
    # get all reviews for product
    nextPage = True
    while nextPage == True:
        get_reviews_single_page(driver, product_filename)  
        nextPage = pagination(driver)
            
    # return to start
    driver.get(url)
    time.sleep(2)
    show_more(driver)
    i += 1
    product = driver.find_elements_by_xpath('.//div[contains(@class,"box listing-item")]')[i]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Starting product: OREO®
Processed 5 reviews
Processed 30 reviews
Processed 30 reviews
Processed 30 reviews
Processed 30 reviews
Processed 30 reviews
Processed 30 reviews
Processed 30 reviews
Processed 1 reviews
16
Starting product: OREO® & CHIPS AHOY!® 2in1
Processed 5 reviews
Processed 30 reviews
Processed 30 reviews
Processed 30 reviews
Processed 14 reviews
17
Starting product: Vanilla Caramel
Processed 5 reviews
Processed 30 reviews
Processed 2 reviews
18
Starting product: Vanilla Caramel Gelato Indulgences
Processed 5 reviews
Processed 30 reviews
Processed 30 reviews
Processed 20 reviews
19
Starting product: Chocolate Mint
Processed 5 reviews
Processed 30 reviews
Processed 3 reviews
20
Starting product: Peach
Processed 5 reviews
Processed 30 reviews
Processed 14 reviews
21
Starting product: Vanilla Fudge Twirl
Processed 5 reviews
Processed 17 reviews
22
Starting product: CarbSmart™ Chocolate
Processed 5 reviews
Processed 30 reviews
Processed 14

IndexError: list index out of range

In [38]:
# # if failed search on product, call this function to remove data before resuming
# def remove_data_pname(product_name):
#     key = "-".join(product_name.encode("ascii", "ignore").decode().strip().split())
    
#     df = pd.read_csv("products_breyers.csv")
#     df[df["key"] != key].to_csv("products_breyers.csv", index=False)
    
#     df = pd.read_csv("reviews_breyers.csv")
#     df[df["key"] != key].to_csv("reviews_breyers.csv", index=False)
    
# def clean_up():
#     with open("last_started.txt", "r") as f:
#         product_name = f.read()
#         remove_data_pname(product_name)

# # print(pd.read_csv("products_breyers.csv").tail(1)["name"].values)
# remove_data_pname("OREO®")

# Scrap work

In [185]:
# initialize driver
driver = webdriver.Chrome('./chromedriver.exe')

# --- get starting page with list of products ---
# define starting page
url = "https://www.haagendazs.us/products/non-dairy/chocolate-fudge-non-dairy-bar"

# load the page
driver.get(url)  
time.sleep(3)

WATER, SUGAR, CORN SYRUP, CHOCOLATE, COCONUT OIL, POWDERED SUGAR, SUGAR, CORN STARCH, COCOA BUTTER, SUNFLOWER OIL, COCOA PROCESSED WITH ALKALI, COCOA, SOYBEAN OIL, SOY LECITHIN, PECTIN, VANILLA EXTRACT, SALT, SUNFLOWER LECITHIN


In [239]:
df2 = pd.read_csv("products_hd.csv")
df2["ingredients"] = df2["ingredients"].apply(lambda x: x.upper().rstrip("."))
df2.to_csv("products_hd_fixed.csv", index=False)

df1 = pd.read_csv("reviews_hd.csv")
df1[~df1.duplicated()].to_csv("reviews_hd_fixed.csv", index=False)

# Remove duplicates and make combined dataset
- For each brand, remove duplicated records in each product and reviews file
- Examine missing values
- Note: Chocolate-Chip-Cookie-Dough is a key that appears twice

In [130]:
p_bj = pd.read_csv("kaggle/bj/products.csv")
p_hd = pd.read_csv("kaggle/hd/products.csv")
p_t = pd.read_csv("kaggle/talenti/products.csv")
p_br = pd.read_csv("kaggle/breyers/products.csv")

r_bj = pd.read_csv("kaggle/bj/reviews.csv")
r_hd = pd.read_csv("kaggle/hd/reviews.csv")
r_t = pd.read_csv("kaggle/talenti/reviews.csv")
r_br = pd.read_csv("kaggle/breyers/reviews.csv")

In [131]:
from shutil import copyfile
p_dfs = [p_bj, p_hd, p_t, p_br]
r_dfs = [r_bj, r_hd, r_t, r_br]
brands = ["bj","hd","talenti","breyers"]
for i,brand in enumerate(brands):
    key_map = {key:str(str(j) + "_" + brand) for j,key in enumerate(p_dfs[i]["key"].values)}
    p_dfs[i]["key"] = p_dfs[i].replace(key_map)
    r_dfs[i]["key"] = r_dfs[i].replace(key_map)
    
    for img_name in next(os.walk("kaggle/" + brand +"/images"))[2]:
        key_name = img_name.split(".png")[0]
        try:
            new_key = key_map[key_name]
        except:
            print(key_name)
        copyfile("kaggle/" + brand + "/images/" + img_name, "kaggle/" + brand + "/images/" + new_key + ".png")
        os.remove("kaggle/" + brand + "/images/" + img_name)
    p_dfs[i].to_csv("kaggle/" + brand + "/products.csv", index=False)
    r_dfs[i].to_csv("kaggle/" + brand + "/reviews.csv", index=False)
    

In [132]:
p_dfs[2].head()

,key,name,description,rating,rating_count,ingredients
0,0_talenti,ALPHONSO MANGO SORBETTO,Our simple and delicious Alphonso Mango Sorbet...,4.7,139,"MANGOS, WATER, SUGAR, DEXTROSE, LEMON JUICE, C..."
1,1_talenti,BANANA CARAMEL CRUNCH,This flavor was inspired by a classic southern...,4.2,25,"MILK, BANANAS, SUGAR†, WHEAT FLOUR, CREAM, NON..."
2,2_talenti,BELGIAN CHOCOLATE GELATO,Our Belgian Chocolate gelato is made with melt...,4.8,27,"MILK, SUGAR, CREAM, EGG AND EGG YOLK, DEXTROSE..."
3,3_talenti,BLACK RASPBERRY CHOCOLATE CHIP GELATO,"Tart and sweet, our Black Raspberry Chocolate ...",4.0,105,"MILK, SUGAR, CREAM, BLACK RASPBERRIES, DEXTROS..."
4,4_talenti,BLACK RASPBERRY VANILLA PARFAIT,This Layer was inspired by one of our favorite...,4.5,65,"MILK, SUGAR, CREAM, BLACK RASPBERRY PUREE, ROL..."


In [76]:
p_dfs = [p_bj, p_hd, p_t, p_br]
r_dfs = [r_bj, r_hd, r_t, r_br]
brands = ["bj","hd","talenti","breyers"]
for i,brand in enumerate(brands):
    print(brand)
    print(p_dfs[i].shape, r_dfs[i].shape)
    if "Chocolate-Chip-Cookie-Dough" in p_dfs[i]["key"].values:
        p_dfs[i].loc[p_dfs[i]["key"]=="Chocolate-Chip-Cookie-Dough","key"] = "Chocolate-Chip-Cookie-Dough_" + brand
    if "Chocolate-Chip-Cookie-Dough" in r_dfs[i]["key"].values:
        r_dfs[i].loc[r_dfs[i]["key"]=="Chocolate-Chip-Cookie-Dough","key"] = "Chocolate-Chip-Cookie-Dough_" + brand

    p_dfs[i] = p_dfs[i][~p_dfs[i].duplicated()]
    r_dfs[i] = r_dfs[i][~r_dfs[i].duplicated()]
    print(p_dfs[i].shape, r_dfs[i].shape)
    
    p_dfs[i].to_csv("products_" + brand + ".csv", index=False)
    r_dfs[i].to_csv("reviews_" + brand + ".csv", index=False)

bj
(57, 7) (7943, 8)
(57, 7) (7943, 8)
hd
(70, 6) (4655, 12)
(70, 6) (4655, 12)
talenti
(45, 6) (4069, 8)
(45, 6) (4069, 8)
breyers
(69, 6) (5009, 8)
(69, 6) (5007, 8)


In [133]:
p_bj = pd.read_csv("kaggle/bj/products.csv")
p_hd = pd.read_csv("kaggle/hd/products.csv")
p_t = pd.read_csv("kaggle/talenti/products.csv")
p_br = pd.read_csv("kaggle/breyers/products.csv")

r_bj = pd.read_csv("kaggle/bj/reviews.csv")
r_hd = pd.read_csv("kaggle/hd/reviews.csv")
r_t = pd.read_csv("kaggle/talenti/reviews.csv")
r_br = pd.read_csv("kaggle/breyers/reviews.csv")

p_dfs = [p_bj, p_hd, p_t, p_br]
r_dfs = [r_bj, r_hd, r_t, r_br]
brands = ["bj","hd","talenti","breyers"]
for i,brand in enumerate(brands):
    brand_col = pd.DataFrame({"brand":[brand]*len(p_dfs[i])})
    p_dfs[i] = pd.concat([brand_col,p_dfs[i]], axis=1)
    brand_col2 = pd.DataFrame({"brand":[brand]*len(r_dfs[i])})
    r_dfs[i] = pd.concat([brand_col2,r_dfs[i]], axis=1)
    
p_concat = pd.concat(p_dfs, axis=0, ignore_index=True)
r_concat = pd.concat(r_dfs, axis=0, ignore_index=True)
p_concat.to_csv("kaggle/combined/products.csv", index=False)
r_concat.to_csv("kaggle/combined/reviews.csv", index=False)

In [139]:
r_concat.dtypes

brand           object
key             object
author          object
date            object
stars            int64
title           object
helpful_yes    float64
helpful_no     float64
text            object
taste          float64
ingredients    float64
texture        float64
likes           object
dtype: object